# gridded_data_tutorial
## Notebook 1
Waterhackweek 2020
Steven Pestana (spestana@uw.edu)
***

### NumPy: working with multi-dimensional arrays in python

In python, the [NumPy](https://numpy.org/) package provides an `ndarray` data type which can be used to represent multi-dimensional gridded data. It also includes linear algebra and other useful math functions. 

See these resources for more detailed NumPy information and tutorials:
* [NumPy: the absolute basics for beginners](https://numpy.org/devdocs/user/absolute_beginners.html)
* [NumPy: creating and manipulating numerical data](https://scipy-lectures.org/intro/numpy/index.html)
* [Advanced NumPy](https://scipy-lectures.org/advanced/advanced_numpy/index.html)
* [NumPy for MATLAB users](https://numpy.org/doc/stable/user/numpy-for-matlab-users.html)

Import NumPy and give it the alias `np` 

In [5]:
import numpy as np

Create a 1-dimensional array:

In [6]:
one_dimensional_array = np.array([0,1,2,3,4,5,6,7,8,9]) 

In [7]:
print(one_dimensional_array)

[0 1 2 3 4 5 6 7 8 9]


We can check the data type of this object, and see that it is an `ndarray`

In [8]:
type(one_dimensional_array)

numpy.ndarray

We can also look at the array's shape

In [75]:
one_dimensional_array.shape

(10,)

Create a 2-dimensional array (this array has three rows and three columns):

In [10]:
two_dimensional_array = np.array([[1,2,3],
                                  [1,2,3],
                                  [1,2,3]])

In [11]:
print(two_dimensional_array)

[[1 2 3]
 [1 2 3]
 [1 2 3]]


In [79]:
two_dimensional_array.shape

(3, 3)

Now create a 3-dimensional array (3-by-3-by-3):

In [80]:
three_dimensional_array = np.array([[[1,2,3],
                                  [1,2,3],
                                  [1,2,3]],
                                  [[1,2,3],
                                  [1,2,3],
                                  [1,2,3]]])

### Index/slicing ndarrays

To select specific elements within an ndarray, you can use slicing, or indexing

The syntax for specifying a **slice** is `x[i:j:k` where for an array `x`, `i` specifies the index to start the slice at, `j` the index to end the slice, and `k` the step size to take moving between `i` and `j`. A step size of 1 does not need to be explicitly stated, it is the default when no step size is provided (`x[i:j]`).

In [81]:
# Starting at the first element (index=0), slice until the fifth element, with a step size of 2
one_dimensional_array[0:5:2]

array([0, 2, 4])

We can slice through multiple dimensions, separating the slice for each dimension with a comma like `x[i:j:k,l:m:n]` where `i`, `j`, and `k` slice the first dimension, and `l`, `m`, and `n` slice the second dimension.

In [90]:
# Select the first two indices of each dimension form a 3-dimensional array
three_dimensional_array[0:2,0:2,0:2]

array([[[1, 2],
        [1, 2]],

       [[1, 2],
        [1, 2]]])

A single index can also be specified to select a single element from the array.

In [91]:
# Select the single value from the center of this 3x3x3 array
three_dimensional_array[1,1,1]

2

Negative indexes will count backwards from the last element in an array (where the last element has index of -1).

In [92]:
# Select the second-to-last element of this one-dimensional array
one_dimensional_array[-2]

8